In [ ]:
from google.cloud import bigquery
import pandas as pd
import pandas_gbq


In [ ]:
project_id = "cf-mccuagcf-recommenders-ht"
location = "EU"


In [ ]:
client = bigquery.Client(project=project_id, location=location)


In [ ]:
dataset_id = "segmentation"


In [ ]:
existing_query = f"""
        SELECT DISTINCT month_id 
        FROM `{project_id}.{dataset_id}.ml_bi_invoices_tbl`
        ORDER BY month_id DESC
        """


In [ ]:
result = client.query(existing_query).to_dataframe()
print(result.head())


In [ ]:
# Test the main query for the pipeline
test_query = """
SELECT
  invoices.cust_person_id,
  invoices.date_of_day,
  (invoices.art_no * 1000000 + invoices.var_tu_key) AS product_id,
  invoices.sell_val_nsp,
  stores.city
FROM
  `cf-mccuagcf-recommenders-ht.segmentation.ml_bi_invoices_tbl` AS invoices
LEFT JOIN
  `cf-mccuagcf-recommenders-ht.segmentation.ml_bi_stores_tbl` AS stores
ON
  invoices.store_id = stores.store_id
WHERE
  invoices.month_id = 202506
  AND invoices.flag_cust_target_group = "SCO"
  AND sell_val_nsp > 0
  AND city = "TERNOPIL"
LIMIT 1000
"""

test_result = client.query(test_query).to_dataframe()
print(f"Query returned {len(test_result)} rows")
print(test_result.head())


In [ ]:
# Verify data types and structure
print(f"Data types:\n{test_result.dtypes}")
print(f"Unique users: {test_result['cust_person_id'].nunique()}")
print(f"Unique products: {test_result['product_id'].nunique()}")
